In [30]:
import sqlite3
import wget
from ufal.udpipe import Model, Pipeline

Подключаемся к уже созданное базе данных с упоминаниями и предложениями

In [181]:
conn = sqlite3.connect('mentions.db')
c = conn.cursor()

Подготавливаем к работе Udpipe, который будет производить токенизацию, лемматизацию, парсинг и разметку частей речи 

In [32]:
udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
modelfile = wget.download(udpipe_url)
model = Model.load(modelfile)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

100% [........................................................................] 40616122 / 40616122

In [33]:
# создадим функцию, которая будет парсить заданные на вход строки
def parsing(text):
    # обрабатываем текст, получаем результат в формате conllu:
    processed = process_pipeline.process(text)
     # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]
    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]
    return tagged

Удалим из базы данных все строчки, где именная группа является одиночным местоимением, потому что эти данные не подойдут для поставленной задачи и исследуемых признаков

In [34]:
c.execute('SELECT Именная_группа FROM mentions')
clmn_mentions = c.fetchall()  #  столбец базы данных, являющая списком кортежей
all_mentions = []  #  список для упоминаний
for x in clmn_mentions: 
    mention = x[0]
    all_mentions.append(mention) 
    #  преобразуем столбец в список упоминаний:
    mention_tg = parsing(mention)
    for token in mention_tg:
        if token[3] == 'PRON'and len(mention_tg)==1:  # проверка, что именная группа состоит из одного местоимения
            # удаляем такие упоминания из базы данных
            c.execute('DELETE FROM mentions WHERE Именная_группа=?', (mention,))
            conn.commit()


1. Длина именной группы

In [176]:
import numpy as np 


np_length = []  #  будущий список для длин каждой именной группы в словах
for mention in all_mentions:
    np_length.append(len(mention.split()))  #  добавляем длину каждой именной группы в список
mean_length = np.mean(np.array(np_length))  #  рассчитываем среднюю длину
# сравниваем длину каждой именной группы со средним показателем и на основе этого формируем бинарный признака
feature_1 = []
for num in np_length:
    if num>mean_length:
        feature_1.append(1)
    else:
        feature_1.append(0)


2. Позиция ремы

In [66]:
c.execute('SELECT Предложение FROM mentions')
clmn_sentences = c.fetchall() # столбец предложений из базы данных
all_sentences = [] # будущий список для предложений
feature_2 = [] # будущий вектор признака
# преобразуем список кортежей в список строк
for x in clmn_sentences:
    sentence = x[0]
    all_sentences.append(sentence) 
for i in range(len(all_sentences)):
    center = len(all_sentences[i])//2  # находим середину предложения
    mention_start = all_sentences[i].find(all_mentions[i])  # индекс первого символа упоминания в данном предложении
    mention_finish = mention_start + len(all_mentions[i])  # индекс последнего символа упоминания в данном предложении
    # в зависимости от трех возможных расположений именной группы относительно центра формируем признаковый вектор
    if mention_start<center and mention_finish<center:
        x = 0
    elif mention_start<center and mention_finish>center:
        x = 1
    elif mention_start>center and mention_finish>center:
        x = 2
    feature_2.append(x)

3. Позиция в тексте

In [182]:
feature_3 = []
c.execute('SELECT Файл FROM mentions') 
clmn_files = c.fetchall() # столбец из базы данных с номерами файлов, из которых взяты предложения
# для каждого предложения и текста, из которого оно взято, смотрим на позицию
for i in range(len(all_sentences)):
    text_num = clmn_files[i][0]
    pathtext = './ForTraining/Texts/book_%d.txt' % text_num
    with open(pathtext, encoding='utf-8') as g:  # открывается сам текст
        text = g.read()
    sentences = text.splitlines()  #  текст делится на предожения
    text_to_3 = len(sentences)//3*3  # делим предложения текста на 3 части
    third1 = sentences[0:text_to_3//3]
    third2 = sentences[text_to_3//3:2*text_to_3//3]
    third3 = sentences[2*text_to_3:]
    # проверка вхождения данного предложения в первую, вторую и третью треть текста
    if all_sentences[i] in third1:
        x = 2
    elif all_sentences[i] in third2:
        x = 1
    elif all_sentences[i] in third3:
        x = 0
    feature_3.append(x)
    
        
        

 4. Позиция подлежащего

In [185]:
feature_4 = [] # будущий вектор признака
for i in range(len(all_mentions)):
    x = 0
    mention_prs = parsing(all_mentions[i]) # обрабатываем udpipe каждую именную группу
    sentence_prs = parsing(all_sentences[i]) # обрабатываем udpipe каждое предложение
    # находим в именной группе вершину:
    for token in mention_prs: 
        if token[7] == 'root':
            mention_root = token[1]
    words = all_sentences[i].split() # предложение, поделенное на слова
    for j in range(len(words)):
        if words[j] == mention_root:
            mention_pos = j  # найдена позиция вершины именной группы в предложении
            if sentence_prs[mention_pos][7]=='nsubj': # проверка, что именно вершина ИГ является подлежащим
                x = 1
    feature_4.append(x)

5. Количество прилагательных в именной группе

In [186]:
adnum_length = []  #  будущий список для кол-ва прилагательных в каждой ИГ
for mention in all_mentions:
    adnum = 0
    mention_prs = parsing(mention)
    for token in mention_prs: # среди разбитого на токены упоминания ищем прилагательные
        if token[3] == 'ADJ':
            adnum += 1
    adnum_length.append(adnum) #  добавляем кол-во прилагательных в данной ИГ в список
mean_num = np.mean(np.array(adnum_length))  #  рассчитываем среднее количество
# сравниваем кол-во прилагательных каждой ИГ со средним показателем и на основе этого формируем бинарный признака
feature_5 = []
for num in adnum_length:
    if num>mean_num:
        feature_5.append(1)
    else:
        feature_5.append(0)


6. Имена собственные в именной группе

In [187]:
import re

feature_6 = []
for mention in all_mentions:
    mention_prs = parsing(mention)
    prop_names = 0 # кол-во имен собственных в ИГ
    brackets = 0  # кол-во названий с кавычками в ИГ
    foreign = 0 # кол-во иностранных слов в ИГ
    abbr = 0  # кол-во аббревиатур в ИГ
    for toren in mention_prs:
        if token[3] == 'PROPN':  # проверка на имя собственное
            prop_names += 1
        if token[7] == 'flat:foreign': # проверка на иностранное слово
            foreign += 1
    match_brackets = re.search(r'"[А-Яа-я ]+"', mention) # поиск названий в кавычках
    if match_brackets:
        brackets += 1
    match_abbr = re.search('[А-Я]+', mention) # поиск аббревиатур
    if match_abbr:
        abbr += 1
    x = 0
    # проверка выполнения хотя бы одного из условий, чтобы можно было назвать первым упоминанием
    if len(mention_prs)>1 and prop_names>1 or brackets>0 or foreign>0 or abbr>0 and len(mention_prs)>1:
        x = 1
    feature_6.append(x)

7. Рестриктивные постмодификации

In [188]:
feature_7 = []
for mention in all_mentions:
    x = 0
    mention_prs = parsing(mention)  
    for token in mention_prs:
        if 'acl' in token[7]: # поиск соответствующего тега среди размеченной ИГ
            x = 1
    feature_7.append(x)        

8. Приложения 

In [189]:
feature_8 = []
for mention in all_mentions:
    x = 0
    mention_prs = parsing(mention)
    for token in mention_prs:
        if token[7] == 'appos': # поиск соответствующего тега среди размеченной ИГ
            x = 1
    feature_8.append(x)


9. Квазибытийные глаголы

In [144]:
feature_9 = []
with open('списки слов\глаголы.txt', encoding='utf-8') as f:
    verbs = f.readlines()
# по категориям запишем файл в списки:
exist1 = verbs[0].split() 
exist2 = verbs[1].split()
speech = verbs[2].split()
for i in range(len(all_mentions)):
    sentence_prs = parsing(all_sentences[i])
    mention_prs = parsing(all_mentions[i])
    x1,x2 = 0, 0 # показатели, которые понадобятся при выводе итогового значения признака
    # нахождение вершины ИГ:
    for token in mention_prs:
        if token[7] == 'root':
            mention_root = token[1]
    # для каждого предложения проверим выполнение условий
    for j in range(len(sentence_prs)):
        # если подлежащее - вершина ИГ, а сказуемое - глагол из exist1
        if sentence_prs[j][1] == mention_root and sentence_prs[j][7]=='nsubj':
            x1 = 1
        if sentence_prs[j][2] in exist1 and sentence_prs[j][7]=='root':
                x2 = 1
        # если вершина ИГ - прямое дополнение глагола из exist2
        if sentence_prs[j][1] == mention_root and sentence_prs[j][7]=='obj':
            x1 = 1
        if sentence_prs[j][2] in exist2 and sentence_prs[j][7]=='root':
            x2 = 1
        # если вершина ИГ является зависимым предикатов речи из speech
        if sentence_prs[j][1] == mention_root and sentence_prs[j][7]=='obl':
            x1 = 1
        if sentence_prs[j][2] in speech and sentence_prs[j][7]=='root':
            x2 = 1
    # необходимо одновременное выполнение x1 и x2, поэтому используем произведение
    feature_9.append(x1*x2)

        
                

10. Модификаторы

In [157]:
feature_10 = []
with open('списки слов\прилагательные.txt', encoding='utf-8') as f:
    modifiers = f.read().split()
for mention in all_mentions:
    x = 0
    mention_prs = parsing(mention)
    # проверка каждого токена ИГ на вхождение в составленный список слов или на превосходную степень прилагательного
    for token in mention_prs:
        if token[2] in modifiers or 'Degree=Sup' in token[5]:
            x = 1
    feature_10.append(x)

11. Неопределенные маркеры 

In [159]:
feature_11 = []
with open('списки слов\неопределенные маркеры.txt', encoding='utf-8') as f:
    markers = f.read().split()
for mention in all_mentions:
    x = 0
    mention_prs = parsing(mention)
    # проверка каждого токена ИГ на вхождение в составленный список слов
    for token in mention_prs:
        if token[2] in modifiers:
            x = 1
    feature_11.append(x)

12. Маркеры неравноправия

In [165]:
feature_12 = []
with open('списки слов\маркеры неравноправия.txt', encoding='utf-8') as f:
    markers = f.read().split()
for mention in all_mentions:
    x = 0
    mention_prs = parsing(mention)
    # проверка каждого токена ИГ на вхождение в составленный список слов
    for token in mention_prs:
        if token[2] in modifiers:
            x = 1
    feature_12.append(x)

13. Маркеры схожести

In [169]:
feature_13 = []
with open('списки слов\маркеры схожести.txt', encoding='utf-8') as f:
    markers = f.read().split()
for mention in all_mentions:
    x = 0
    mention_prs = parsing(mention)
    # проверка каждого токена ИГ на вхождение в составленный список слов
    for token in mention_prs:
        if token[2] in modifiers:
            x = 1
    feature_13.append(x)

14. Предикаты, комплементом которых служат новые упоминания

In [173]:
feature_14 = []
with open('списки слов\предикаты.txt', encoding='utf-8') as f:
    predicates = f.read().split()
for mention in all_mentions:
    x = 0
    mention_prs = parsing(mention)
    # проверка каждого токена ИГ на вхождение в составленный список слов
    for token in mention_prs:
        if token[2] in predicates:
            x = 1
    feature_14.append(x)

15. Объекты, существование которых точно известно 

In [174]:
feature_15 = []
with open('списки слов\объекты.txt', encoding='utf-8') as f:
    objects = f.read().split()
for mention in all_mentions:
    x = 0
    mention_prs = parsing(mention)
    # проверка каждого токена ИГ на вхождение в составленный список слов
    for token in mention_prs:
        if token[2] in objects:
            x = 1
    feature_15.append(x)

Создадим Dataframe со всеми полученными признаками, на котором будем обучать и тестировать

In [193]:
import pandas as pd

df = pd.DataFrame({
    'Длина ИГ': feature_1,
    'Позиция ремы': feature_2,
    'Позиция в тексте': feature_3,
    'Подлежащее': feature_4,
    'Кол-во прил': feature_5,
    'Имена соб-е': feature_6,
    'Рестрикт': feature_7,
    'Приложения': feature_8,
    'Каузация существ.': feature_9,
    'Модификаторы': feature_10,
    'Неопр маркеры': feature_11,
    'Маркеры неравн': feature_12,
    'Маркеры схож': feature_13,
    'Предикаты': feature_14,
    'Объекты': feature_15
}, index = all_mentions)
    

Вот так выглядят все рассматриваемые именные группы, размеченные по 15 признакам: 

In [197]:
df

,Длина ИГ,Позиция ремы,Позиция в тексте,Подлежащее,Кол-во прил,Имена соб-е,Рестрикт,Приложения,Каузация существ.,Модификаторы,Неопр маркеры,Маркеры неравн,Маркеры схож,Предикаты,Объекты
﻿«Школа злословия»,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0
«Школа злословия»,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0
сценаристка Дуня Смирнова,1,0,2,0,0,1,0,1,0,0,0,0,0,0,0
канале «Культура»,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0
ведущих,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0
«Культуры»,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
ведущих,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
Татьяне Никитичне,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
Татьяны Толстой,0,2,1,0,1,1,0,0,0,0,0,0,0,0,0
египтологом Виктором Солкиным,1,0,1,0,0,1,0,1,1,0,0,0,0,0,0


In [195]:
df.to_csv('FFFM.csv')  # сохраняем в файл таблицу

In [196]:
conn.close()  # отключаемся от базы данных